#### Modelo WM para el banano del día martes

In [ ]:
options(warm=-1)  # no mostrar mensajes de alerta
library(frbs) # libreria para generar las redes neuronales
library(readxl)   # libreria para leer datos de excel

In [ ]:
t <- proc.time() # Inicia el cronómetro

RMSE1<-matrix(0, 1, 2) # RMSE[numero producto]
MAPE1<-matrix(0, 1, 2)

trainbm <- data_Bam[1 : 237, 1: 4] # cambiar numero por j
testbm <- data_Bam[238 : 296, 1 : 4] # cambiar numero por j
realbm <- data_Bam[293 : 307, 5]
bamy <- data_Bam[238 : 296, 4]

attach(testbm)
b<-Bamy 

for (j in 1:2) {
  
  # Define el intervalo de los resultados a obtener
  rango <-apply(trainbm[, j: 4], 2, range)
  
  # Paraemtros del modelo para ANFIS
  control <- list(num.labels = 3, type.mf= "GAUSSIAN",  step.size = 0.01, type.tnorm = "MIN", type.snorm = "MAX",
                  type.implication.func = "ZADEH")
  
  # Modelo
  object.reg <- frbs.learn(trainbm[,j: 4], rango, method.type="WM", control)
  
  # Graficas de las funciones de pertenencia para las variables
  plotMF(object.reg)
  
  ## Caracteristicas del modelo 
  summary(object.reg)
  
  last_one<- nrow(testbm[, j: 4])
  
  ##########################################
  flag_1<-0
  stop_1<-0
  paso_1<-1
  
  while (stop_1==0) {
    if(b[paso_1]!=0)
    {
      flag_1=flag_1+1
    } else{
      stop_1=1
    }
    paso_1=paso_1+1
  }
  ##########################################
  
  contbm1<-flag_1
  
  compbm2<- testbm[last_one, length(testbm)]
  while(compbm2 ==0){
    Bamy_r<-(predict(object.reg, within(testbm[,j:4], rm(Bamy))))
    
    if (contbm1<last_one) {
      testbm[contbm1+1,3]<- Bamy_r[contbm1]
      testbm[contbm1+1,2]<- Bamy_r[contbm1-1,]
      testbm[contbm1+1,1]<- Bamy_r[contbm1-10,]
    }
    
    contbm1<-contbm1+1
    testbm[contbm1,length(testbm)]<-Bamy_r[contbm1]
    compbm2<- testbm[last_one, length(testbm)]
  }
  
  b<-bamy 
  
  Bamy_r[last_one]<-(predict(object.reg, within(testbm[last_one,j:4], rm(Bamy))))
  
  # Error RMSE y MAPE
  RMSEbm<- sqrt(sum((realbm-Bamy_r[45:59])^2)/(15))
  MAPEbm<- sum(((abs(realbm- Bamy_r[45:59]))/realbm)*100)/(15)
  
  RMSE1[1, j]<-RMSEbm
  MAPE1[1, j]<-MAPEbm
  
  trainbm <- data_Bam[1 : 237, 1: 4] # cambiar numero por j
  testbm <- data_Bam[238 : 296, 1 : 4] # cambiar numero por j
  realbm <- data_Bam[293 : 307, 5]
  bamy <- data_Bam[238 : 296, 4]
  
  b<-Bamy 
  
  
}
proc.time()-t